In [1]:
import torch
from d2l import torch as d2l

In [2]:
X, W_xh = torch.normal(0, 1, (3, 1)), torch.normal(0, 1, (1, 4)) # 3 * 4
H, W_hh = torch.normal(0, 1, (3, 4)), torch.normal(0, 1, (4, 4))
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[-0.2063, -2.1521, -0.4828,  3.5992],
        [-0.2784,  5.6078,  1.7859, -2.6562],
        [ 0.1244,  1.8829, -2.0657,  7.0816]])

In [7]:
tmp = torch.ones((1, 4))
torch.matmul(X, tmp)

tensor([[-0.2523, -0.2523, -0.2523, -0.2523],
        [ 0.4248,  0.4248,  0.4248,  0.4248],
        [ 2.7612,  2.7612,  2.7612,  2.7612]])

In [5]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0)) # 拼接，

tensor([[-1.3630,  1.2467, -2.1987, -0.6945],
        [-1.3832,  2.4258,  0.9499, -1.7862],
        [ 1.9049,  2.8829,  1.0527,  0.3457]])

In [6]:
torch.cat((X, H), 1)

tensor([[-1.6060, -1.7461, -1.5597,  1.0724, -0.4283],
        [-0.1285,  1.1895,  0.6861,  0.8491,  0.7461],
        [ 0.2367,  0.4611,  2.1900, -0.0726,  0.6052]])

In [8]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [9]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps) # 一个分词列表和词表, 统计频率和词 等等

In [25]:
train_iter?

Type:           SeqDataLoader
String form:    <d2l.torch.SeqDataLoader object at 0x7ff7326e2450>
File:           ~/anaconda3/envs/torch/lib/python3.11/site-packages/d2l/torch.py
Docstring:      An iterator to load sequence data.
Init docstring: Defined in :numref:`sec_language_model`

In [10]:
F.one_hot(torch.tensor([0, 1, 2]), 5)

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0]])

In [11]:
vocab

In [12]:
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, 28)

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0

- 初始化模型参数

In [12]:
def get_params(vocab_size, num_hiddens, device): # 单层 隐藏层参数和隐变量参数 隐变量和输入，在一开始就需要运算隐变量
    
    num_inputs = num_outputs = vocab_size
    
    def normal(shape): # 生成正太分布
        return torch.randn(size=shape, device=device) * 0.01
    # 前三个
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h= torch.zeros(num_hiddens, device=device)
    # 得到值后再同意计算
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [13]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), ) # 返回元组

In [14]:
# 构建特殊的隐藏层 可是我该怎么
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量⼤⼩，词表⼤⼩)
    W_xh, W_hh, b_h, W_hq, b_q = params # 随机身材
    H, = state
    outputs = []
    # X的形状：(批量⼤⼩，词表⼤⼩)
    for X in inputs: # 激活函数
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h) # 初始为全零状态，然后再跟着输入改变
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

In [15]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经⽹络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
        get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn
        
        
    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params) # RNN
    
    
    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [38]:
len(vocab)

28

In [18]:
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
    init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

RuntimeError: one_hot is only applicable to index tensor.